# DIY Covid-19 Dashboard

The following dashboard aims to successfully implement the following functionalities:


1. Acquire pandemic data from the Public Health England API through the Software Development Kit
2.  Visualisation and wrangling of data in the form of at least one graph
3.  Enable a user to trigger the API access by clicking on a button, and providing at least a functional interactive control (in the form of a widget) for one of the graphs


In [36]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [37]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [38]:
data = {}
with open("agedistribution.json", "rt") as INFILE:
    data=json.load(INFILE)

## Part 1 and 2: Aquiring Data and Visualisation

The following dashboard represents the number of COVID-19 cases for males and females based on their age range. 
The first step was to access data through the Public Health England through a web-based Application Programming Interface (API). This often involves the excange of information in JSON format. 
Public Health Englanda offers a Python Software Devekopment Kit (SDK). This is a Python wrapper that facilitates access to the API by building requests. 

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [39]:
def wrangle_data(rawdata):
    
    datadic=rawdata['data'][0]

    males=datadic['males']
    females=datadic['females']

    ageranges=[x['age'] for x in males] 

    def min_age(agerange):
        agerange=agerange.replace('+','') 
        start=agerange.split('_')[0]
        return int(start)

    ageranges.sort(key=min_age)
    age_df=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])

    for entry in males: 
        ageband=entry['age'] 
        age_df.loc[ageband, 'males']=entry['value']

    for entry in females:
        ageband=entry['age']
        age_df.loc[ageband, 'females']=entry['value']

    age_df['total']=age_df['males']+age_df['females']

    return age_df

age_df=wrangle_data(data)

## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [40]:
  
def access_api():
    filters = [
    'areaType=nation',
    'areaName=England'
    ]

    structure = {
    "males": "maleCases",
    "females": "femaleCases"
    }
    api = Cov19API(filters=filters, structure=structure)
    agedistribution=api.get_json()
    return agedistribution




def api_button_callback(button):
    apidata=access_api()
    global age_df
    age_df=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"

    
apibutton=wdg.Button(
    description='Refresh data', 
    disabled=False,
    button_style='', 
    tooltip="'Click to download current Public Health England data'",
    icon='download'
)
    



AGE DISTRIBUTION GRAPH

This interactive graph represents the number of COVID-19 cases in England for males and females by dividing the data based on their age range. By adding an interactive control provided by iPywidgets, allows to select any combination of the males, females or total columns.


Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [41]:
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], # options available
    value=['males'], # initial value
    rows=3, # rows of the selection box
    description='Sex',
    disabled=False
)

def refresh_graph():
    current=agecols.value[0]
    if current==agecols.options[0]:
        other=agecols.options[1]
    else:
        other=agecols.options[0]
    agecols.value=(other,) # forces the redraw
    agecols.value=(current,) # now we can change it back


def age_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        age_df.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in variable output    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols})

apibutton.on_click(api_button_callback) 
display(apibutton)

display(agecols, output)

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip="'Click to download current P…

SelectMultiple(description='Sex', index=(0,), options=('males', 'females', 'total'), rows=3, value=('males',))

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*